### Binomial Asset Pricing Model
Implementation of a slow and fast binomial pricing model in python. We first treat the binomial tree as a network of nodes (i, j), with i representing the time steps and j representing the number of ordered price outcome (bottom of tree -> lowest, top of tree -> highest)


In [10]:
import numpy as np

#### Timing Wrapper Function
This function will be used to benchmark the two binomial models

In [11]:
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r args:[%r, %r] took: %2.4f sec' % \
              (f.__name__, args, kw, te-ts))
        return result
    return wrap

### Representation of the Binomial Tree
Stock price can be represented using nodes (i, j) and initial stock price $ S_{0} $

$ S_{ij} = S_{0}u^{j}d^{i-j} $

$ C_{ij} $ represents the price of the call option at each node (i, j). Where $ C_{Nj} $ represents the final payoff function that we will define

For this project, we shall price a European Call, so $ C_{Nj} $ = max($ S_{Nj} $ - K, 0)



In [12]:
# Initialise parameters
S0 = 100    # initial stock price
K = 100     # strike price
T = 1       # time to maturity in years
r = 0.06    # common risk-free interest rate
N = 3       # number of time steps
u = 1.1     # up factor
d = 1/u     # down factor, to ensure recombining tree
opt = 'C'   # Option Type 'C' or 'P'

### Binomial Tree (Slow)
We use loops to iterate through nodes j at each time step i

In [13]:
@timing
def binomial_tree_slow(S0, K ,T , r, N, u, d, opt = 'C'):
    # constants
    dt = T/N    # each time step
    q = (np.exp(r*dt)-d) / (u-d)  # risk-neutral probability
    disc = np.exp(-r*dt)

    # initialise asset price at maturity - Time step N
    S = np.zeros(N+1)  # N step binomial tree will result in N+1 nodes at i = N
    S[0] = S0*d**N     # Initialising bottom of the tree at the end
    for j in range(1,N+1):
        S[j] = S[j-1]*u/d

    # initialise option values at maturity
    C = np.zeros(N+1)
    for j in range(0, N+1):
        C[j] = max(S[j] - K, 0)

    # moving backwords through the tree
    for i in np.arange(N,0,-1):     # each time period
        for j in range(0, i):       # each position of tree
            C[j] = disc * (q*C[j+1] + (1-q)*C[j]) # iterative process

    return C[0]
binomial_tree_slow(S0, K ,T , r, N, u, d, opt = 'C')

func:'binomial_tree_slow' args:[(100, 100, 1, 0.06, 3, 1.1, 0.9090909090909091), {'opt': 'C'}] took: 0.0010 sec


10.145735799928817

### Binomial Tree Fast
Now we will vectorise the code above using numpy arrays instead of using for loops through j nodes 

In [14]:
@timing
def binomial_tree_fast(S0, K ,T , r, N, u, d, opt = 'C'):
    # constants
    dt = T/N    # each time step
    q = (np.exp(r*dt)-d) / (u-d)  # risk-neutral probability
    disc = np.exp(-r*dt)

    # initialise asset price at maturity - Time step N
    C = S0 * d **(np.arange(N,-1,-1)) * u **(np.arange(0, N+1, 1))  # C is an array containing the asset prices at maturity

    # initialise option values at maturity
    C = np.maximum(C - K, 0)    # C returns an array containing max value between C-K and 0 (iterates through C)
    
    # moving backwards through the tree
    for i in np.arange(N, 0 , -1):
        C = disc * ( q * C[1:i+1] + (1-q) * C[0:i])  #C[1:i+1] is the up vector, C[0:i] is the down vector
    
    return C[0]
binomial_tree_fast(S0, K ,T , r, N, u, d, opt = 'C')

func:'binomial_tree_fast' args:[(100, 100, 1, 0.06, 3, 1.1, 0.9090909090909091), {'opt': 'C'}] took: 0.0000 sec


10.145735799928826

### Binomial Tree Model Fast vs Slow
We now compare the runtimes for the models above, assuming all parameters kept constant except for N

In [16]:
for N in [10, 100, 1000, 5000]:
    binomial_tree_slow(S0, K ,T , r, N, u, d, opt = 'C')
    binomial_tree_fast(S0, K ,T , r, N, u, d, opt = 'C')

func:'binomial_tree_slow' args:[(100, 100, 1, 0.06, 10, 1.1, 0.9090909090909091), {'opt': 'C'}] took: 0.0010 sec
func:'binomial_tree_fast' args:[(100, 100, 1, 0.06, 10, 1.1, 0.9090909090909091), {'opt': 'C'}] took: 0.0005 sec
func:'binomial_tree_slow' args:[(100, 100, 1, 0.06, 100, 1.1, 0.9090909090909091), {'opt': 'C'}] took: 0.0020 sec
func:'binomial_tree_fast' args:[(100, 100, 1, 0.06, 100, 1.1, 0.9090909090909091), {'opt': 'C'}] took: 0.0000 sec
func:'binomial_tree_slow' args:[(100, 100, 1, 0.06, 1000, 1.1, 0.9090909090909091), {'opt': 'C'}] took: 0.1975 sec
func:'binomial_tree_fast' args:[(100, 100, 1, 0.06, 1000, 1.1, 0.9090909090909091), {'opt': 'C'}] took: 0.0055 sec
func:'binomial_tree_slow' args:[(100, 100, 1, 0.06, 5000, 1.1, 0.9090909090909091), {'opt': 'C'}] took: 4.6301 sec
func:'binomial_tree_fast' args:[(100, 100, 1, 0.06, 5000, 1.1, 0.9090909090909091), {'opt': 'C'}] took: 0.0344 sec
